<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/nbs/models/bert-base-uncased-maxlen320-2epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip

--2020-10-08 12:33:20--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.64.4
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.64.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  12.7MB/s    in 9.3s    

2020-10-08 12:33:31 (10.5 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]



In [3]:
!unzip ./Embold_Participant\'s_Dataset.zip

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Embold_Participant's_Dataset/
  inflating: Embold_Participant's_Dataset/sample submission.csv  
  inflating: __MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Embold_Participant's_Dataset/embold_train_extra.json  
  inflating: __MACOSX/Embold_Participant's_Dataset/._embold_train_extra.json  
  inflating: Embold_Participant's_Dataset/embold_test.json  
  inflating: __MACOSX/Embold_Participant's_Dataset/._embold_test.json  
  inflating: Embold_Participant's_Dataset/embold_train.json  
  inflating: __MACOSX/Embold_Participant's_Dataset/._embold_train.json  


In [2]:
cd Dataset/

/content/Dataset


In [3]:
ls

 embold_test.json          embold_train.json
 embold_train_extra.json  'sample submission.csv'


In [4]:
#import pandas as pd
import numpy as np
import pandas as pd

In [5]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
train_df.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [6]:
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
test_df.head()

,title,body
0,config question path-specific environment var...,issue description or question\r \r hey @artemg...
1,crash indien vol,de simulator crasht als hij vol zit
2,unable to mine rocks,"sarkasmo starting today, when i hit enter act..."
3,not all whitelists are processed,create following rules... order of creation is...
4,add ctx menu for idafree 70 and idafree 5,"associated with .dll, .dll_, .exe, .exe_, .sc,..."


In [7]:
train_ex_df = pd.read_json("embold_train_extra.json")
train_ex_df.head()

,title,body,label
0,use a 8bit typeface,since this is meant to emulate some old arcade...,1
1,implement wireless m-bus binding,_from chris.pa...@googlemail.com https://cod...,1
2,add multilang support for timeago.js,currently it is only en . \r required to add ...,1
3,scaleway - seg-fault on shutdown,tbr irc creates a new scaleway instance with...,0
4,sistema de pintura: no se guardar los nuevos p...,este sp ya estaba asignado a un carro y se enc...,0


In [8]:
train_ex_df.shape, train_df.shape, test_df.shape

((300000, 3), (150000, 3), (30000, 2))

In [9]:
train_data = train_df.append(train_ex_df)

In [10]:
test_df.shape, train_data.shape

((30000, 2), (450000, 3))

In [11]:
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

## Model Training

In [17]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 1.1MB 24.1MB/s 
     |████████████████████████████████| 3.0MB 25.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=81653d1b5dc5d9139b921ef977dabb3dca13a9c45a75f99d6ba962c426a41566
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import TFAutoModelWithLMHead, AutoTokenizer
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [13]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "bert-base-uncased"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.80.149.66:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.80.149.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.80.149.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [15]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [16]:
%%time

X_data = regular_encode(train_data.text.values.tolist()[0:100000], tokenizer, maxlen=transformer_flags.MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 57 s, sys: 3.24 s, total: 1min
Wall time: 34.2 s


In [17]:
X_data.shape

(100000, 320)

In [18]:
y_train = train_data.label.values

In [19]:
input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL)
output_layer = bert_model(input_layer)
output_layer

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


(<tf.Tensor 'tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 320, 768) dtype=float32>,
 <tf.Tensor 'tf_bert_model/bert/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>)

In [21]:
output = tf.keras.layers.Dense(3, activation='softmax')(output_layer[1])
output

<tf.Tensor 'dense/Softmax:0' shape=(None, 3) dtype=float32>

In [22]:
tf.keras.backend.clear_session()

In [23]:
with strategy.scope():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL)
  output_layer = bert_model(input_layer)
  output = tf.keras.layers.Dense(3, activation='softmax')(output_layer[1])
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 320, 768), (None, 109482240 
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [24]:
classifier_model.fit(X_data, y_train[0:100000], epochs=3, batch_size=global_batch_size, validation_split=0.1)

Epoch 1/3
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  2/704 [..............................] - ETA: 3:54:02 - loss: 1.0850 - accuracy: 0.4805WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0102s vs `on_train_batch_end` time: 0.3080s). Check your callbacks.


704/704 [==============================] - ETA: 0s - loss: 0.5320 - accuracy: 0.7923WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_test_batch_end` time: 0.0861s). Check your callbacks.


704/704 [==============================] - 317s 451ms/step - loss: 0.5320 - accuracy: 0.7923 - val_loss: 0.4591 - val_accuracy: 0.8329
Epoch 2/3
704/704 [==============================] - 229s 326ms/step - loss: 0.4384 - accuracy: 0.8379 - val_loss: 0.4484 - val_accuracy: 0.8338
Epoch 3/3
 76/704 [==>...........................] - ETA: 3:18 - loss: 0.4264 - accuracy: 0.8426

KeyboardInterrupt: ignored

## Test Prediction and submission

In [25]:
%%time

X_test_data = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
X_test_data.shape

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 16.9 s, sys: 324 ms, total: 17.2 s
Wall time: 9.65 s


In [26]:
y_test_pred = classifier_model.predict(X_test_data, batch_size=global_batch_size, verbose=1)

  2/235 [..............................] - ETA: 15:59WARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0036s vs `on_predict_batch_end` time: 0.0863s). Check your callbacks.


235/235 [==============================] - 34s 146ms/step


In [27]:
y_test_pred[0]

array([0.01295947, 0.24729101, 0.7397495 ], dtype=float32)

In [28]:
y_test_pred_label = y_test_pred.argmax(axis=1)

In [30]:
y_test_pred_label.shape

(30000,)

In [31]:
submission = pd.read_csv('sample submission.csv')

In [33]:
submission.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [34]:
submission['label'] = y_test_pred_label

In [35]:
submission.to_csv('bert-base-uncased-maxlen320-2epochs.csv', index=False)